# Critical Role stats

* Longest stretch of time without speaking

## Download all Campaign 2 subtitles

(takes about an hour)

```bash
youtube-dl https://www.youtube.com/playlist\?list\=PL1tiwbzkOjQxD0jjAE7PsWoaCrs0EkBH2 \
    --skip-download \
    --write-sub \
    --sub-lang en \
    --convert-subs srt
```

In [1]:
import os
import re
import datetime
from collections import defaultdict
import webvtt
from IPython.display import display, Markdown
import humanfriendly
from pandas import DataFrame as df

/home/rodrigo/.pyenv/versions/3.8.6/envs/critrolesubs/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
os.path.abspath(os.curdir)

'/home/rodrigo/critrolesubs'

In [3]:
title_pattern = r'(?P<filename>^(?P<title>.+) _ Critical Role ?_ Campaign 2,? Episode (?P<episode>\d+).*$)'
speaker_pattern = r'(?P<speaker>^[A-Z]+):'
emotion_pattern = r'^[\(\[](?P<emotion>.+)[\)\]]$'

In [4]:
def parse_subtitles_directory(path):
    '''
    "path" is a directory containing subtitles in VTT format
    for Campaign 2, downloaded with youtube-dl. Returns a list
    of dictionaries containing { title, filename, episode } (with
    `episode` being the episode number in the campaign), sorted
    by episode number
    '''
    filenames = [sub for sub in os.listdir(path) if sub.lower().endswith('.vtt')]
    parsed = [
        re.match(title_pattern, fn).groupdict()
        for fn in filenames
    ]
    return sorted(parsed, key=lambda x: int(x['episode']))

episodes = parse_subtitles_directory('.')
episodes[0]

{'filename': 'Curious Beginnings  _ Critical Role _ Campaign 2, Episode 1-byva0hOj8CU.en.vtt',
 'title': 'Curious Beginnings ',
 'episode': '1'}

In [5]:
def get_episode_subtitles(filename):
    parser = webvtt.webvtt.WebVTT()
    return parser.read(filename).captions

In [7]:
subtitles = get_episode_subtitles(episodes[0]['filename'])
subtitles[0:5]

[<Caption start=00:00:00.100 end=00:00:01.354 text=MATT: Hello, everyone.>,
 <Caption start=00:00:01.354 end=00:00:03.170 text=(laughter)>,
 <Caption start=00:00:03.170 end=00:00:07.071 text=MATT: And welcome to tonight's episode of Critical\nRole, where a bunch of us nerdy-ass voice actors>,
 <Caption start=00:00:07.071 end=00:00:12.030 text=sit around and play the first game of our new\ncampaign of Dungeons and Dragons.>,
 <Caption start=00:00:12.030 end=00:00:15.002 text=(cheering)>]

In [10]:
CAST = ['Matt', 'Travis', 'Marisha', 'Liam', 'Sam', 'Laura', 'Taliesin', 'Ashley', 'All', 'Everyone', 'Brian']

In [11]:
def get_lines_per_cast_member(subtitles):
    lines_per_person = defaultdict(list)
    last_subtitle = None

    for subtitle in subtitles:
        speaker = re.match(speaker_pattern, subtitle.text)

        if speaker is None:
            is_emotion = bool(re.match(emotion_pattern, subtitle.text))
            if is_emotion:
                lines_per_person['All'].append(subtitle)
            else:
                last_subtitle._lines += subtitle.lines
                last_subtitle._end = subtitle._end
            continue
        else:
            last_subtitle = subtitle

        speaker_name = speaker.groupdict()['speaker'].title()
        lines_per_person[speaker_name].append(subtitle)

        if speaker_name not in CAST:
            print(f'Added to cast: {speaker_name}')
            CAST.append(speaker_name)
    
    return dict(lines_per_person)

In [12]:
lines_per_person = get_lines_per_cast_member(subtitles)
lines_per_person['Matt'][0:3]

[<Caption start=00:00:00.100 end=00:00:01.354 text=MATT: Hello, everyone.>,
 <Caption start=00:00:03.170 end=00:00:12.030 text=MATT: And welcome to tonight's episode of Critical\nRole, where a bunch of us nerdy-ass voice actors\nsit around and play the first game of our new\ncampaign of Dungeons and Dragons.>,
 <Caption start=00:00:15.002 end=00:01:21.948 text=MATT: Super excited to be back. I've missed this a\nlot. Missed this-- oh, you guys will be fine. So\nyes. Glad you guys can join us, apparently in some\ncrazy numbers. So welcome new viewers. Hope you\nenjoy your stay. So, without further ado, let's\nget this going so we can get through our\nannouncements. Since it is our first episode of\nthe year, of the campaign, I want to make sure\nthat we get our announcements out of the way.\nWe've got some cool stuff to talk about. First and\nforemost, we have two sponsors tonight. Our first\nsponsor-- and we're going into, for a time, a cool\npartnership with the folks at DnD Beyond,\na

In [13]:
def get_intervals_between_lines(lines):
    intervals = []

    for index, line in enumerate(lines):
        if index == 0:
            continue
        previous_line = lines[index - 1]
        interval = (
            datetime.timedelta(seconds=line.start_in_seconds)
            - datetime.timedelta(seconds=previous_line.end_in_seconds)
        )
        intervals.append({
            'interval': interval,
            'starting_line': previous_line,
            'ending_line': line
        })

    return sorted(intervals, key=lambda x: x['interval'], reverse=True)

def get_intervals_between_lines_per_speaker(lines_per_person):
    intervals = {}

    for speaker, lines in lines_per_person.items():
        intervals[speaker] = get_intervals_between_lines(lines)
    
    return intervals

In [14]:
intervals_per_speaker = get_intervals_between_lines_per_speaker(lines_per_person)
intervals_per_speaker['Matt'][0:3]

[{'interval': datetime.timedelta(seconds=1017, microseconds=119000),
  'starting_line': <Caption start=01:32:08.081 end=01:32:09.582 text=MATT: We'll see you here in a few minutes.>,
  'ending_line': <Caption start=01:49:06.701 end=01:49:10.388 text=MATT: Welcome back, ladies and gentlemen. Guys!\nWow!>},
 {'interval': datetime.timedelta(seconds=173, microseconds=214000),
  'starting_line': <Caption start=00:22:04.811 end=00:22:21.919 text=MATT: So Caleb, as you come to consciousness, you\nglance over and can see, slowly snoring and\nrousing at about the same time, Nott's eyes blink\nopen, her slowly groaning face looking over\ntowards you.>,
  'ending_line': <Caption start=00:25:15.133 end=00:25:25.268 text=MATT: At this point, you notice the gentle smell\nof cooked meats and promises of mediocre porridge\nand eggs just barely begins to creep beneath the\nsubtle floor of your room, meet your nostrils.>},
 {'interval': datetime.timedelta(seconds=134, microseconds=115000),
  'starting_l

In [15]:
def get_longest_interval_per_speaker(intervals_per_speaker):
    longest_intervals = {}
    for speaker, intervals in intervals_per_speaker.items():
        if intervals:
            longest_intervals[speaker] = intervals[0]
    return longest_intervals

In [16]:
longest_intervals = get_longest_interval_per_speaker(intervals_per_speaker)
longest_intervals

{'Matt': {'interval': datetime.timedelta(seconds=1017, microseconds=119000),
  'starting_line': <Caption start=01:32:08.081 end=01:32:09.582 text=MATT: We'll see you here in a few minutes.>,
  'ending_line': <Caption start=01:49:06.701 end=01:49:10.388 text=MATT: Welcome back, ladies and gentlemen. Guys!\nWow!>},
 'All': {'interval': datetime.timedelta(seconds=427, microseconds=258000),
  'starting_line': <Caption start=02:24:25.904 end=02:24:28.715 text=(yelling)>,
  'ending_line': <Caption start=02:31:35.973 end=02:31:40.504 text=(laughter)>},
 'Laura': {'interval': datetime.timedelta(seconds=1205, microseconds=298000),
  'starting_line': <Caption start=01:32:10.442 end=01:32:11.302 text=LAURA: Magic!>,
  'ending_line': <Caption start=01:52:16.600 end=01:52:31.389 text=LAURA: Here's the thing. Some of us have weapons\nthat we don't want to pass away to other people,\nso how much of a chance is it that we could get\nthrough without having to do that, you know?>},
 'Travis': {'interval

In [17]:
def get_longest_interval_of_episode(intervals_per_speaker):
    longest_intervals = get_longest_interval_per_speaker(intervals_per_speaker)
    return sorted(longest_intervals.items(), reverse=True, key=lambda x: x[1]['interval'])[0]

def get_speaker_with_longest_interval(intervals_per_speaker):
    return get_longest_interval_of_episode(intervals_per_speaker)[0]

In [18]:
speaker, longest_interval = get_longest_interval_of_episode(intervals_per_speaker)
speaker, longest_interval

('Ashley',
 {'interval': datetime.timedelta(seconds=1766, microseconds=284000),
  'starting_line': <Caption start=00:56:06.954 end=00:56:10.331 text=ASHLEY: Yep, nice to meet you all. You're so, uh,\nwonderful.>,
  'ending_line': <Caption start=01:25:36.615 end=01:25:37.698 text=ASHLEY: Beedle-deedle.>})

In [18]:
episodes = parse_subtitles_directory('.')

for episode in episodes:
    display(Markdown(f'# Episode {episode["episode"]} - {episode["title"]}'))
    
    subtitles = get_episode_subtitles(episode['filename'])
    # print duration, number of lines
    lines_per_person = get_lines_per_cast_member(subtitles)
    intervals_per_speaker = get_intervals_between_lines_per_speaker(lines_per_person)
    longest_intervals = get_longest_interval_per_speaker(intervals_per_speaker)
    
    display(
        df(longest_intervals).transpose().sort_values(by='interval', ascending=False)
    )
    
    

# Episode 1 - Curious Beginnings 

,interval,starting_line,ending_line
Ashley,0 days 00:29:26.284000,"00:56:06.954 00:56:10.331 ASHLEY: Yep, nice to...",01:25:36.615 01:25:37.698 ASHLEY: Beedle-deedle.
Taliesin,0 days 00:28:24.553000,00:16:28.205 00:16:30.509 TALIESIN: This has b...,00:44:55.062 00:44:57.204 TALIESIN: Neither di...
Marisha,0 days 00:21:55.364000,01:30:04.248 01:30:05.810 MARISHA: What?,01:52:01.174 01:52:04.532 MARISHA: What happen...
Laura,0 days 00:20:05.298000,01:32:10.442 01:32:11.302 LAURA: Magic!,01:52:16.600 01:52:31.389 LAURA: Here's the th...
Travis,0 days 00:19:01.754000,01:31:04.635 01:31:06.904 TRAVIS: Face is on f...,01:50:08.658 01:50:10.238 TRAVIS: Bring that a...
Sam,0 days 00:18:54.369000,"01:31:10.886 01:31:15.869 SAM: Ashley, I'll do...",01:50:10.238 01:50:14.875 SAM: Five dragons sw...
Liam,0 days 00:18:40.496000,02:05:35.398 02:05:38.825 LIAM: I'm going to k...,02:24:19.321 02:24:23.861 LIAM: The miniatures...
Matt,0 days 00:16:57.119000,01:32:08.081 01:32:09.582 MATT: We'll see you ...,"01:49:06.701 01:49:10.388 MATT: Welcome back, ..."
All,0 days 00:07:07.258000,02:24:25.904 02:24:28.715 (yelling),02:31:35.973 02:31:40.504 (laughter)


# Episode 2 - A Show of Scrutiny

,interval,starting_line,ending_line
All,0 days 00:32:10.831000,03:23:07.433 03:23:10.215 (laughter),03:55:21.046 03:55:25.773 (laughter)
Travis,0 days 00:20:39.095000,03:36:20.674 03:36:21.687 TRAVIS: You ever cas...,03:57:00.782 03:57:02.949 TRAVIS: Reversal!
Taliesin,0 days 00:19:24.878000,02:05:37.518 02:05:41.150 TALIESIN: I'd like t...,02:25:06.028 02:25:13.838 TALIESIN: I'm compar...
Sam,0 days 00:19:16.673000,02:05:58.521 02:06:01.514 SAM: This is excitin...,02:25:18.187 02:25:21.534 SAM: Your game is re...
Liam,0 days 00:19:12.490000,02:05:21.170 02:05:26.979 LIAM: It's been a wh...,02:24:39.469 02:24:42.860 LIAM: That's your wo...
Laura,0 days 00:17:44.632000,02:04:39.270 02:04:40.480 LAURA: What?,02:22:25.112 02:22:26.905 LAURA: Do we still h...
Marisha,0 days 00:15:20.516000,02:05:41.150 02:05:42.161 MARISHA: Agreed.,02:21:02.677 02:21:07.322 MARISHA: I love how ...
Matt,0 days 00:14:13.539000,02:06:01.514 02:06:28.138 MATT: Indeed. I supp...,"02:20:41.677 02:21:02.563 MATT: Welcome back, ..."


# Episode 3 - The Midnight Chase

...

# Episode 109 - Frigid Propositions

Added to cast: Lauar


,interval,starting_line,ending_line
Ashley,0 days 00:27:02.777000,03:11:43.247 03:11:47.758 ASHLEY: But you know...,03:38:50.535 03:38:55.887 ASHLEY: For clarific...
Marisha,0 days 00:26:30.026000,02:00:17.499 02:00:21.478 MARISHA: You're the ...,02:26:51.504 02:26:52.438 MARISHA: (imitating ...
Taliesin,0 days 00:25:35.727000,00:39:02.375 00:39:03.384 TALIESIN: That was a...,"01:04:39.111 01:04:40.676 TALIESIN: No, it's o..."
Sam,0 days 00:21:36.500000,02:03:03.648 02:03:07.157 SAM: A Kiwi goliath?...,02:24:43.657 02:24:46.252 SAM: It's a ring of ...
Laura,0 days 00:20:00.084000,02:01:01.427 02:01:07.363 LAURA: I pat Fjord o...,02:21:07.447 02:21:08.810 LAURA: Oh my goodness.
Liam,0 days 00:19:52.367000,02:01:35.955 02:01:40.590 LIAM: Amazing.\nDrag...,02:21:32.957 02:21:34.577 LIAM: Oh my-\nMARISH...
All,0 days 00:18:44.295000,"02:05:58.629 02:06:01.957 (soft, dark music)",02:24:46.252 02:24:49.692 (laughter)
Travis,0 days 00:18:18.660000,"02:03:07.157 02:03:08.687 TRAVIS: Oh, fucking ...","02:21:27.347 02:21:28.937 TRAVIS: Yeah, it's j..."
Matt,0 days 00:12:41.260000,02:03:08.687 02:08:06.307 MATT: So take a shor...,02:20:47.567 02:21:07.447 MATT: And welcome ba...
Brian,0 days 00:00:15.515000,"00:05:07.897 00:05:55.778 BRIAN: Hey gang, but...","00:06:11.293 00:06:12.935 BRIAN: Bye, love you!"


# Episode 110 - Dinner with the Devil

,interval,starting_line,ending_line
Ashley,0 days 00:30:33.842000,02:55:46.346 02:55:47.187 ASHLEY: Yes.,03:26:21.029 03:26:22.756 ASHLEY: That's a han...
Sam,0 days 00:29:59.444000,00:18:30.725 00:18:32.125 SAM: Take your time....,"00:48:31.569 00:48:32.801 SAM: Well, one's alive."
All,0 days 00:28:29.054000,00:08:35.731 00:08:40.673 (flames whooshing),00:37:09.727 00:37:12.061 (laughter)
Taliesin,0 days 00:26:08.886000,01:57:38.961 01:57:39.794 TALIESIN: Hmm.,02:23:48.680 02:23:51.256 TALIESIN: Is this ha...
Travis,0 days 00:21:59.501000,02:34:21.335 02:34:22.509 TRAVIS: Oh no.,02:56:22.010 02:56:23.563 TRAVIS: This insight...
Laura,0 days 00:20:08.734000,02:01:48.551 02:02:06.106 LAURA: Out of the oa...,02:22:14.840 02:22:25.680 LAURA: Can I look to...
Liam,0 days 00:19:51.423000,02:00:27.531 02:00:29.227 LIAM: The ball.\nSAM...,02:20:20.650 02:20:54.270 LIAM: 15.\nMATT: 15?...
Matt,0 days 00:16:59.575000,02:01:40.146 02:01:41.725 MATT: Ambition's Call.,02:18:41.300 02:20:16.483 MATT: Welcome back.\...
Marisha,0 days 00:14:32.185000,02:04:45.131 02:06:26.735 MARISHA: Hello every...,02:20:58.920 02:20:59.940 MARISHA: 31.


In [19]:
pprint = lambda x: display(Markdown(x))

In [25]:
pprint("## Matt's times")
for episode in episodes:
    subtitles = get_episode_subtitles(episode['filename'])
    pprint(f'### Episode {episode["episode"]} - {episode["title"]}')
    lines_per_person = get_lines_per_cast_member(subtitles)
    intervals_per_speaker = get_intervals_between_lines_per_speaker(lines_per_person)
    longest_intervals = get_longest_interval_per_speaker(intervals_per_speaker)
    matt = df(longest_intervals)['Matt']

    pprint(f"**Interval**: {humanfriendly.module.format_timespan(matt['interval'].seconds)}")
    pprint(f'**Starting line**: {matt["starting_line"].start} {matt["starting_line"].text}')
    pprint(f'**Ending line**: {matt["ending_line"].start} {matt["ending_line"].text}')

## Matt's times

### Episode 1 - Curious Beginnings 

**Interval**: 16 minutes and 57 seconds

**Starting line**: 01:32:08.081 MATT: We'll see you here in a few minutes.

**Ending line**: 01:49:06.701 MATT: Welcome back, ladies and gentlemen. Guys!
Wow!

...

**Ending line**: 02:20:47.567 MATT: And welcome back.
So, as you all stand now in the center
of the Endless Emporium of Enchanted Eccentricities,
you see who you assume to be Thaydeen there,
arms out on the edges of the central counter, going,
"So, what are you looking for, if I might ask, huh?"

### Episode 110 - Dinner with the Devil

**Interval**: 16 minutes and 59 seconds

**Starting line**: 02:01:40.146 MATT: Ambition's Call.

**Ending line**: 02:18:41.300 MATT: Welcome back.
So. The gates (creaking)
open before you.
The courtyard, leading to the tower.
As you walk through, it's a beautiful,
if slightly ominous, patch of forest surrounding this tower.
You can see faint fire bugs glowing
and fading in and out in the distance,
but just far enough out of reach,
as you walk this paved path that leads through.
Eventually, you find yourself to the base of the tower
where this massive set of arched double doors
that wait for you open into a low-lit antechamber.
As you walk into the tower chamber proper,
you are met with a patterned carpet of dark brown and red
that meets wall to wall.
No furniture, just an open entry.
You can see a beautiful crystal chandelier
that hangs above you in the center
of the partially domed ceiling.
You can see intricately carved wooden panels
wrap around the doorless walls, aside from the entryway,
housing large paintings set within elaborate frames.
You can see one large landscape that shows an odd sunrise,
it seems, over a desolate plain of rock and flame,
a sky of burning clouds and haze.
Anyone here who is trained in arcana, if you'd like to roll?

In [110]:
episodes = parse_subtitles_directory('.')

for episode in episodes:
    episode['subtitles'] = get_episode_subtitles(episode['filename'])

In [111]:
line_count = []

for episode in episodes:
    # print duration, number of lines
    lines_per_person = get_lines_per_cast_member(episode['subtitles'])
    
    episode_counts = {}
    episode_counts['title'] = episode['title']
    episode_counts['episode'] = episode['episode']
    for person, lines in lines_per_person.items():
#         episode_counts[person] = len(lines)
          episode_counts[person] = sum([len(x.text) for x in lines_per_person[person]])
    line_count.append(episode_counts)


In [113]:
line_count = df(line_count)

In [107]:
line_count[['episode', 'title', 'Matt']].sort_values(by='Matt')

,episode,title,Matt
48,49,A Game of Names,51344
76,77,A Tangled Web,57834
36,37,Dangerous Liaisons,58049
90,91,Stone to Clay,59568
91,92,Home Is Where the Heart Is,61468
...,...,...,...
24,25,Divergent Paths,110675
54,55,Duplicity,111005
6,7,Hush,113136
85,86,The Cathedral,117997


In [93]:
counts = counts.reindex(columns=['episode', 'title', 'Matt'])

In [95]:
counts[['episode', 'title', 'Matt']].sort_values(by='Matt')

,episode,title,Matt
36,37,Dangerous Liaisons,378
29,30,The Journey Home,378
0,1,Curious Beginnings,397
56,57,In Love and War,410
48,49,A Game of Names,421
...,...,...,...
38,39,Temple of the False Serpent,1069
97,98,Dark Waters,1108
81,82,The Beat of the Permaheart,1112
54,55,Duplicity,1171


In [32]:
cast = counts['cast']

In [34]:
cast.axes

[RangeIndex(start=0, stop=109, step=1)]

In [35]:
cast.transpose()

0      {'Matt': 397, 'All': 112, 'Laura': 283, 'Travi...
1      {'Matt': 658, 'Travis': 259, 'Taliesin': 288, ...
2      {'Matt': 777, 'All': 59, 'Liam': 306, 'Sam': 2...
3      {'Matt': 542, 'All': 51, 'Sam': 204, 'Travis':...
4      {'Matt': 659, 'Laura': 288, 'All': 36, 'Travis...
                             ...                        
104    {'Matt': 535, 'Travis': 308, 'Ashley': 154, 'A...
105    {'Matt': 609, 'All': 129, 'Taliesin': 216, 'Sa...
106    {'Matt': 487, 'All': 123, 'Sam': 189, 'Laura':...
107    {'Matt': 560, 'All': 126, 'Sam': 161, 'Ashley'...
108    {'Matt': 476, 'All': 94, 'Sam': 204, 'Laura': ...
Name: cast, Length: 109, dtype: object

In [36]:
cast

0      {'Matt': 397, 'All': 112, 'Laura': 283, 'Travi...
1      {'Matt': 658, 'Travis': 259, 'Taliesin': 288, ...
2      {'Matt': 777, 'All': 59, 'Liam': 306, 'Sam': 2...
3      {'Matt': 542, 'All': 51, 'Sam': 204, 'Travis':...
4      {'Matt': 659, 'Laura': 288, 'All': 36, 'Travis...
                             ...                        
104    {'Matt': 535, 'Travis': 308, 'Ashley': 154, 'A...
105    {'Matt': 609, 'All': 129, 'Taliesin': 216, 'Sa...
106    {'Matt': 487, 'All': 123, 'Sam': 189, 'Laura':...
107    {'Matt': 560, 'All': 126, 'Sam': 161, 'Ashley'...
108    {'Matt': 476, 'All': 94, 'Sam': 204, 'Laura': ...
Name: cast, Length: 109, dtype: object

In [38]:
cast.to_frame()

,cast
0,"{'Matt': 397, 'All': 112, 'Laura': 283, 'Travi..."
1,"{'Matt': 658, 'Travis': 259, 'Taliesin': 288, ..."
2,"{'Matt': 777, 'All': 59, 'Liam': 306, 'Sam': 2..."
3,"{'Matt': 542, 'All': 51, 'Sam': 204, 'Travis':..."
4,"{'Matt': 659, 'Laura': 288, 'All': 36, 'Travis..."
...,...
104,"{'Matt': 535, 'Travis': 308, 'Ashley': 154, 'A..."
105,"{'Matt': 609, 'All': 129, 'Taliesin': 216, 'Sa..."
106,"{'Matt': 487, 'All': 123, 'Sam': 189, 'Laura':..."
107,"{'Matt': 560, 'All': 126, 'Sam': 161, 'Ashley'..."
